# Get top options with good `sdMult` and `rom`

In [14]:
MARKET = 'NSE'

In [15]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [16]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder

In [17]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = DATAPATH.joinpath('temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [18]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID
PAPER = ibp.PAPER

In [19]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

array(['df_chains.pkl', 'df_ohlcs.pkl', 'df_opts.pkl', 'df_symlots.pkl',
       'df_und_margins.pkl', 'df_unds.pkl', 'qopts.pkl'], dtype='<U18')

In [40]:
# imports
from ib_insync import LimitOrder
from support import get_openorders

In [36]:
# prepare the best to-dos

# ..sort by the rom
df = df_opts.sort_values('rom', ascending = False)

# ... expected rom and sdMult mask
m = (df.rom > ibp.MINEXPROM) & np.where(df.right == 'P', df.sdMult > ibp.PUTSTDMULT, df.sdMult > ibp.CALLSTDMULT)

# ... right mask
r = (df.strike > df.undPrice) & (df.right == 'C') | (df.strike < df.undPrice) & (df.right == 'P')
df1 = df[m & r].sort_values(['right', 'strike'], ascending=[True, False])

#.. add expPrice
df1['expPrice'] = df1.price+0.05

In [44]:
# ... check open ordes and remove them
df_openords = get_openorders(MARKET)
df1 = df1[~df1.conId.isin(df_openords.conId)].reset_index(drop=True)

In [45]:
# ... build the SELL orders
contracts = df1.contract.to_list()
orders = [LimitOrder(action='SELL', totalQuantity=abs(int(q)), lmtPrice=p) 
                        for q, p in zip(df1.qty, df1.expPrice)]

cos = [(c, o) for c, o in zip(contracts, orders)]

# Place the trades

"'df_opt_prices.pkl'[:-4]"